# Группировка данных в DataFrame

In [1]:
import pandas as pd
from IPython.display import display
melb_df = pd.read_csv('data/melb_data_fe.csv')
melb_df['Date'] = pd.to_datetime(melb_df['Date'])
# Сделаем преобразование столбцов к типу данных category
cols_to_exclude = ['Date', 'Rooms', 'Bedroom', 'Bathroom', 'Car'] # список столбцов, которые мы не берём во внимание
max_unique_count = 150 # задаём максимальное число уникальных категорий
for col in melb_df.columns: # цикл по именам столбцов
    if melb_df[col].nunique() < max_unique_count and col not in cols_to_exclude: # проверяем условие
        melb_df[col] = melb_df[col].astype('category') # преобразуем тип столбца

Одна из основных задач анализа данных — это группировка данных и сравнение показателей в группах. Например, нам необходимо сравнить средний уровень заработной платы в зависимости от пола/уровня образования. Или же мы хотим проследить, какая группа клиентов приносит нам наибольший доход, чтобы направить своё внимание на эту группу.

##### МЕТОД GROUPBY()

В библиотеке Pandas для группировки данных по одному или нескольким признакам можно использовать метод groupby().

Основные параметры метода groupby():

* by — имя или список имён столбцов, по которым производится группировка.
* axis — ось, по которой производится группировка (0 — строки, 1 — столбцы). По умолчанию группировка производится по строкам.
* as_index — добавляется ли дополнительный индекс к таблице. По умолчанию установлен на True.

Метод groupby() возвращает объект DataFrameGroupBy, который хранит в себе информацию о том, какие строки относятся к определённой группе, и сам по себе не представляет для нас интереса.

Однако к этому объекту можно применять уже знакомые нам агрегирующие методы (mean, median, sum и т. д.), чтобы рассчитывать показатели внутри каждой группы.

![Общая схема работы метода groupby() в Pandas ](https://lms.skillfactory.ru/assets/courseware/v1/09fc73c845fb48dc9dc3faab5a960f20/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/dst3-u1-md12_3_1.png "groupby()")

Сначала мы разделяем данные на группы с помощью метода groupby(), после чего к каждой группе применяем агрегацию и объединяем результаты в новую таблицу.


##### ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С ОДНОЙ АГРЕГАЦИЕЙ

In [3]:
# Применим агрегирующую функцию среднего к результату работы groupby().
# В качестве столбца для группировки возьмём столбец типа объекта недвижимости (Type)
melb_df.groupby(by='Type', as_index=False).mean()

,Type,Rooms,Price,Distance,Postcode,Bedroom,Bathroom,Car,Landsize,BuildingArea,Lattitude,Longtitude,Propertycount,MeanRoomsSquare,AreaRatio,AgeBuilding
0,house,3.260874,1.242665e+06,10.979479,3104.080643,3.229336,1.613822,1.772674,617.181924,152.162553,-37.803795,144.994700,7259.025505,18.996731,-0.490031,55.669700
1,townhouse,2.837522,9.337351e+05,9.851346,3100.777379,2.814183,1.809695,1.555655,279.606822,134.649710,-37.815782,144.996489,7094.459605,18.569847,-0.094916,26.690305
2,unit,1.963871,6.051275e+05,7.607391,3110.797481,1.966523,1.183295,1.128936,477.314219,102.235863,-37.823710,144.996363,8199.280080,21.068242,0.319883,39.703016


Мы получили таблицу, на пересечении строк и столбцов которой находятся средние значения каждого числового признака в наших данных, а на месте индексов стоят значения типа объекта недвижимости Type (house, townhouse, unit).

Если мы хотим видеть тип объекта в качестве отдельного столбца таблицы, мы можем выставить параметр as_index на False

In [4]:
# Сравним средние цены на объекты в зависимости от их типа
melb_df.groupby('Type')['Price'].mean()

Type
house        1.242665e+06
townhouse    9.337351e+05
unit         6.051275e+05
Name: Price, dtype: float64

В результате мы получаем объект Series

Из этой маленькой таблицы видно, что наибольшей средней ценой обладают объекты типа house (дома, коттеджи, виллы). Следовательно, можно сделать вывод, что тип постройки является значимым фактором при определении цены объекта недвижимости.

In [5]:
# Теперь давайте выясним, какие регионы (Regionname) наиболее удалены от центра Мельбурна
# Найдём минимальное значение расстояния от центра города до объекта в зависимости от его региона
# Результат отсортируем по убыванию расстояния
melb_df.groupby('Regionname')['Distance'].min().sort_values(ascending=False)

Regionname
Western Victoria              29.8
Eastern Victoria              25.2
Northern Victoria             21.8
South-Eastern Metropolitan    14.7
Eastern Metropolitan           7.8
Western Metropolitan           4.3
Southern Metropolitan          0.7
Northern Metropolitan          0.0
Name: Distance, dtype: float64

Итак, наиболее удалёнными являются все регионы Victoria

##### ГРУППИРОВКА ДАННЫХ ПО ОДНОМУ КРИТЕРИЮ С НЕСКОЛЬКИМИ АГРЕГАЦИЯМИ

Чтобы рассчитать несколько агрегирующих методов, можно воспользоваться методом agg(), который принимает список строк с названиями агрегаций

Основные агрегирующие методы:

* .count()      Количество непустых значений  
* .mean()	      Среднее значение  
* .median()     Медиана  
* .min()	      Минимальное значение  
* .max()	      Максимальное значение  
* .deviance()	  Дисперсия  
* .std()	      Стандартное отклонение  
* .sum()	      Сумма  
* .quantile(x)  Квантиль уровня x  
* .nunique()	  Число уникальных значений  

Давайте построим таблицу для анализа продаж по месяцам. Для этого найдём количество продаж, а также среднее и максимальное значения цен объектов недвижимости (Price), сгруппированных по номеру месяца продажи (MonthSale). Результат отсортируем по количеству продаж в порядке убывания

In [7]:
melb_df.groupby('MonthSale')['Price'].agg(['count','mean', 'max']).sort_values(by='count', ascending=False)

,count,mean,max
MonthSale,,,
8,1850,1.056371e+06,6500000.0
7,1835,9.314698e+05,9000000.0
5,1644,1.097807e+06,8000000.0
6,1469,1.068981e+06,7650000.0
3,1408,1.146762e+06,5600000.0
4,1246,1.050479e+06,5500000.0
9,1188,1.126349e+06,6400000.0
10,854,1.135970e+06,6250000.0
11,750,1.142503e+06,5050000.0


В результате применения метода agg(), в который мы передали список с названиями интересующих нас агрегирующих функций, мы получаем DataFrame со столбцами count, mean и max, где для каждого месяца рассчитаны соответствующие параметры. Результат сортируем по столбцу count

In [9]:
# Если вам нужна полная информация обо всех основных статистических характеристиках
# внутри каждой группы, вы можете воспользоваться методом agg(),
# передав в качестве его параметра строку 'describe':
melb_df.groupby('MonthSale')['Price'].agg('describe').sort_values(by='count', ascending=False)

,count,mean,std,min,25%,50%,75%,max
MonthSale,,,,,,,,
8,1850.0,1.056371e+06,619617.476541,160000.0,635000.0,892000.0,1310000.0,6500000.0
7,1835.0,9.314698e+05,537390.803161,190000.0,586750.0,800000.0,1150000.0,9000000.0
5,1644.0,1.097807e+06,668492.867996,145000.0,650000.0,905000.0,1371250.0,8000000.0
6,1469.0,1.068981e+06,606010.069052,222000.0,660000.0,900000.0,1325000.0,7650000.0
3,1408.0,1.146762e+06,709573.596867,85000.0,680000.0,945000.0,1400000.0,5600000.0
4,1246.0,1.050479e+06,591892.902979,145000.0,655000.0,905500.0,1298750.0,5500000.0
9,1188.0,1.126349e+06,608734.690742,170000.0,725000.0,980000.0,1360000.0,6400000.0
10,854.0,1.135970e+06,692950.251627,250000.0,652625.0,950000.0,1416500.0,6250000.0
11,750.0,1.142503e+06,703666.227892,240000.0,671750.0,950000.0,1399125.0,5050000.0


После базовых математических функций наиболее частым агрегированием является подсчёт числа уникальных значений. Так, например, мы можем вычислить число уникальных риелторских компаний в зависимости от региона, чтобы понять, в каких регионах конкуренция на рынке недвижимости меньше.

Это можно сделать, передав в параметр метода agg() строку 'nunique'.

Более того, метод agg() поддерживает использование и других функций. Передадим дополнительно встроенную функцию set, чтобы получить множество из агентств недвижимости, которые работают в каждом из регионов

In [10]:
melb_df.groupby('Regionname')['SellerG'].agg(['nunique', set])

,nunique,set
Regionname,,
Eastern Metropolitan,26,"{C21, Miles, Jellis, hockingstuart, RT, Purple..."
Eastern Victoria,11,"{C21, Fletchers, Ray, O'Brien, hockingstuart, ..."
Northern Metropolitan,40,"{C21, Miles, Jellis, hockingstuart, Jas, RT, P..."
Northern Victoria,11,"{Buckingham, YPA, Ray, McDonald, hockingstuart..."
South-Eastern Metropolitan,25,"{C21, O'Brien, Jellis, hockingstuart, Chisholm..."
Southern Metropolitan,38,"{C21, Jellis, O'Brien, hockingstuart, Chisholm..."
Western Metropolitan,34,"{C21, Jellis, O'Brien, hockingstuart, Chisholm..."
Western Victoria,6,"{YPA, Ray, hockingstuart, Raine, other, HAR}"


##### Задачи

Сгруппируйте данные по признаку количества комнат и найдите среднюю цену объектов недвижимости в каждой группе. В качестве ответа запишите количество комнат, для которых средняя цена наибольшая.

In [13]:
melb_df.groupby('Rooms')['Price'].agg(['mean']).sort_values(by='mean')

,mean
Rooms,
1,4.338245e+05
2,7.750812e+05
10,9.000000e+05
3,1.076081e+06
4,1.445282e+06
8,1.602750e+06
6,1.849366e+06
5,1.870260e+06
7,1.920700e+06


Какой регион имеет наименьшую протяжённость по географической широте (Lattitude)?

Для ответа на этот вопрос рассчитайте стандартное отклонение широты для каждого региона.
В качестве ответа запишите название этого региона.

In [14]:
melb_df.groupby('Regionname')['Lattitude'].agg(['std']).sort_values(by='std')

,std
Regionname,
Western Victoria,0.011579
Southern Metropolitan,0.043080
Eastern Metropolitan,0.047890
Northern Metropolitan,0.049639
Western Metropolitan,0.051251
South-Eastern Metropolitan,0.073411
Northern Victoria,0.084455
Eastern Victoria,0.147067


Какая риелторская компания (SellerG) имеет наименьшую общую выручку за период с 1 мая по 1 сентября (включительно) 2017 года?

Для ответа на этот вопрос рассчитайте сумму продаж (Price) каждой компании в заданный период.
Не забудьте перевести даты в формат datetime.

In [16]:
melb_df.head()

,Suburb,Rooms,Type,Price,Method,SellerG,Date,Distance,Postcode,Bedroom,...,Longtitude,Regionname,Propertycount,MeanRoomsSquare,AreaRatio,MonthSale,AgeBuilding,WeekdaySale,StreetType,Weekend
0,Abbotsford,2,house,1480000.0,S,Biggin,2016-03-12,2.5,3067,2,...,144.9984,Northern Metropolitan,4019,25.20,-0.231707,3,46,5,St,1
1,Abbotsford,2,house,1035000.0,S,Biggin,2016-04-02,2.5,3067,2,...,144.9934,Northern Metropolitan,4019,15.80,-0.327660,4,116,5,St,1
2,Abbotsford,3,house,1465000.0,SP,Biggin,2017-04-03,2.5,3067,3,...,144.9944,Northern Metropolitan,4019,18.75,0.056338,4,117,0,St,0
3,Abbotsford,3,house,850000.0,PI,Biggin,2017-04-03,2.5,3067,3,...,144.9969,Northern Metropolitan,4019,15.75,0.145455,4,47,0,other,0
4,Abbotsford,4,house,1600000.0,VB,Nelson,2016-04-06,2.5,3067,3,...,144.9941,Northern Metropolitan,4019,17.75,0.083969,4,2,2,St,0


In [19]:
melb_df[(melb_df['Date']<='2017-09-1')&(melb_df['Date']>='2017-05-1')].groupby('SellerG')['Price'].agg(['sum']).sort_values(by='sum')

,sum
SellerG,
LITTLE,2742000.0
Cayzer,4439000.0
Burnham,4550500.0
Moonee,7328000.0
Thomson,8332000.0
Bells,8656000.0
Alexkarbon,10985000.0
McDonald,14637500.0
Rendina,15422276.0
